In [ ]:
from persor import BVHparser
from scipy.spatial.transform import Rotation
import matplotlib.pyplot as plt
import numpy as np
import copy


In [ ]:
def get_joint_coords(skeleton, joint, coords):
    offset = skeleton[joint]["offset"]
    parent_joint = skeleton[joint]["joint"]
    child_joints = skeleton[joint]["children"]

    if parent_joint == None:
        return

    parent_coord = coords[parent_joint]["coord"]

    current_coord = [
        parent_coord[0] + offset[0],
        parent_coord[1] + offset[1],
        parent_coord[2] + offset[2],
    ]
    coords[joint] = {"coord": current_coord, "parent": parent_joint}

    for child in child_joints:
        get_joint_coords(skeleton, child, coords)


def skelton2coords(skeleton):
    coords = {"root": {"coord": skeleton["root"]["offset"], "parent": None}}
    skeleton["root"]

    child_joints = skeleton["root"]["children"]
    for child in child_joints:
        get_joint_coords(skeleton, child, coords)

    get_joint_coords(skeleton, "root", coords)

    return coords


def rotated_coord(coord, frame):
    coord_copy = copy.deepcopy(coord)
    for joint, data in coord_copy.items():
        if joint == "root":
            continue

        x_rotate = np.deg2rad(frame[f"{joint}_Xrotation"])
        y_rotate = np.deg2rad(frame[f"{joint}_Yrotation"])
        z_rotate = np.deg2rad(frame[f"{joint}_Zrotation"])

        rot = Rotation.from_rotvec(np.array([x_rotate, y_rotate, z_rotate]))
        coord_copy[joint]["coord"] = rot.apply(data["coord"])

    return coord_copy

In [ ]:
bvhp = BVHparser("./data/jump.bvh")


In [ ]:
skeleton = bvhp.get_skeleton()
motion_df = bvhp.get_motion_df()
motion_frame = motion_df.iloc[100, :]

coords = skelton2coords(skeleton)
coord_rotated = rotated_coord(skeleton, motion_frame)


In [ ]:
import matplotlib.pyplot as plt
import japanize_matplotlib

In [ ]:
def plot_skeleton(coords, title="骨格"):
    fig = plt.figure()
    ax = fig.add_subplot(projection="3d")
    for joint, coord in coords.items():
        if coord["parent"] == None:
            continue
        parent_coord = coords[coord["parent"]]["coord"]
        ax.plot(
            [parent_coord[0], coord["coord"][0]],
            [parent_coord[2], coord["coord"][2]],
            [parent_coord[1], coord["coord"][1]],
            marker="o",
        )
    plt.title(title)
    plt.xlabel("x[m]")
    plt.ylabel("y[m]")
    plt.ylabel("z[m]")
    ax.set_xlim(-90, 90)
    ax.set_ylim(-90, 90)
    ax.set_zlim(0, 190)

    plt.show()

In [ ]:
plot_skeleton(coords, "元の骨格")
plot_skeleton(coord_rotated, "回転後の骨格")

In [ ]:
print(coords)